In [1]:
import pandas as pd
import cx_Oracle
import yaml


from table_mapping import mappings

from pid_tester import tools

In [2]:
ns = {'wfs': 'http://www.opengis.net/wfs/2.0',
      'er': 'http://xmlns.earthresourceml.org/EarthResource/2.0',
     'erl': 'http://xmlns.earthresourceml.org/earthresourceml-lite/1.0'}

In [3]:
with open("/home/michael/Developer/database.yml", 'r') as stream:
    credentials = yaml.load(stream)

In [4]:
oraprod = credentials['oraprod']

In [ ]:
connection = cx_Oracle.connect(user=oraprod["user"],password=oraprod["password"],dsn=oraprod["dsn"])


In [ ]:
for mapping in mappings:
    table = mapping['table']
    query = tools.get_query(table)
    print(query)
    df = pd.read_sql(query, connection)
    pids = mapping['pids']
    for pid, feature_type in pids.items():
        print("Resolving {0} ...".format(pid))
        resolve_column = '{0}_CHECK'.format(pid)
        df[resolve_column] = df[pid].apply(lambda x: tools.resolve_pid(x,feature_type,ns)).apply(pd.Series)
    vocabs = mapping['vocabs']
    #for vocab in vocabs:
    #    resolve_column = '{0}_CHECK'.format(vocab)
    #    df[resolve_column] = df[vocab].apply(tools.resolve_vocab(vocab))
    filename = "{0}.xlsx".format(table)
    print("Saving {0} ...".format(table))
    if 'SHAPE' in df.columns:
        df.drop(['SHAPE'], axis = 1,inplace=True)
    df.to_excel(filename)   
    

select * from MINRES.erl_mineraloccurrenceview
Resolving IDENTIFIER ...
Resolving MINE_URI ...
Resolving SPECIFICATION_URI ...
Saving erl_mineraloccurrenceview ...
select * from MINRES.erl_commodityresourceview
Resolving IDENTIFIER ...
Resolving MINERALOCCURRENCE_URI ...
